**Download the Dataset**

In [ ]:
!gdown "https://drive.google.com/uc?id=1O4-mRTgHK4AntMNDdQmOP-jnxXs9agnQ&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1O4-mRTgHK4AntMNDdQmOP-jnxXs9agnQ&confirm=t
To: /content/tennis_court_det_dataset.zip
100% 7.26G/7.26G [01:47<00:00, 67.8MB/s]


In [ ]:
!unzip "tennis_court_det_dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: data/images/JNKp7sCdQlY_2200.png  
  inflating: data/images/JNKp7sCdQlY_2250.png  
  inflating: data/images/JNKp7sCdQlY_250.png  
  inflating: data/images/JNKp7sCdQlY_300.png  
  inflating: data/images/JNKp7sCdQlY_350.png  
  inflating: data/images/JNKp7sCdQlY_400.png  
  inflating: data/images/JNKp7sCdQlY_450.png  
  inflating: data/images/JNKp7sCdQlY_50.png  
  inflating: data/images/JNKp7sCdQlY_500.png  
  inflating: data/images/JNKp7sCdQlY_550.png  
  inflating: data/images/JNKp7sCdQlY_600.png  
  inflating: data/images/JNKp7sCdQlY_650.png  
  inflating: data/images/JNKp7sCdQlY_700.png  
  inflating: data/images/JNKp7sCdQlY_750.png  
  inflating: data/images/juXbdW7z0WA_100.png  
  inflating: data/images/juXbdW7z0WA_1050.png  
  inflating: data/images/juXbdW7z0WA_1100.png  
  inflating: data/images/juXbdW7z0WA_200.png  
  inflating: data/images/juXbdW7z0WA_350.png  
  inflating: data/images/juXbdW7z0WA_400.png  
  infl

**Import All the Required Libraries**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

**Choose Device**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cpu')

**Create Torch Dataset**

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)

**Create Model**

In [ ]:
model = models.resnet50(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features, 14 * 2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


**Move the Model to the Device**

In [ ]:
model = model.to(device)

**Train the Model**

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [ ]:
epochs=50
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14244.68359375
Epoch 0, iter 10, loss: 14543.7041015625
Epoch 0, iter 20, loss: 13650.517578125
Epoch 0, iter 30, loss: 14030.8642578125
Epoch 0, iter 40, loss: 13343.619140625
Epoch 0, iter 50, loss: 13222.3369140625
Epoch 0, iter 60, loss: 12664.3349609375
Epoch 0, iter 70, loss: 12113.822265625
Epoch 0, iter 80, loss: 12411.8623046875
Epoch 0, iter 90, loss: 11664.3017578125
Epoch 0, iter 100, loss: 11559.98046875
Epoch 0, iter 110, loss: 10293.25
Epoch 0, iter 120, loss: 9871.798828125
Epoch 0, iter 130, loss: 10517.828125
Epoch 0, iter 140, loss: 10093.8125
Epoch 0, iter 150, loss: 9563.6435546875
Epoch 0, iter 160, loss: 9305.9951171875
Epoch 0, iter 170, loss: 8040.826171875
Epoch 0, iter 180, loss: 7964.0068359375
Epoch 0, iter 190, loss: 7656.208984375
Epoch 0, iter 200, loss: 7792.2490234375
Epoch 0, iter 210, loss: 7728.93603515625
Epoch 0, iter 220, loss: 7098.0146484375
Epoch 0, iter 230, loss: 7172.091796875
Epoch 0, iter 240, loss: 6553.89404296875

In [ ]:
torch.save(model.state_dict(), "keypoints_model_50.pth")
